In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import Dataset
import pandas as pd
from torch.optim import Adam
from torch.utils.data import DataLoader
import numpy as np
import tqdm
import torch
from sklearn.model_selection import train_test_split

In [3]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backend.mps.is_available() else "cpu"

In [4]:
path = "/content/Bing_Bing_plot.csv"

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
prompt = ''
def context( prompt, title, summary, speaker, text):
    return f'{prompt} in the episode {title} the plot {summary} {speaker} said {text}'


df =  pd.read_csv(path)
text_columns = ['utterance_by_other' , 'utterance_by_chandler', 'Episode_Title', 'Summary']
df[text_columns] = df[text_columns].astype('str')
text_inputs = ['Episode_Title', 'Summary', 'Interacted_with', 'utterance_by_other']
df['utterance_with_context'] = df[text_inputs].apply(lambda x : context(prompt,  x[0], x[1] , x[2], x[3]), axis = 1)
df['utterance_by_chandler'] = df['utterance_by_chandler'].apply(lambda x: f'chandler said {x}'  )
df_final = df[['utterance_with_context', 'utterance_by_chandler']]

print(df_final['utterance_with_context'][155], '\n',df_final['utterance_by_chandler'][155] )
df_final.head(3)
df_train, df_test = train_test_split(df_final  , test_size = 0.10 , random_state = 1)

 in the episode The One with the Butt the plot Monica's obsessiveness is put to the test after Rachel cleans the apartment. Joey lands a film role as Al Pacino's butt double. Chandler enjoys a relationship with all of the fun but none of the responsibility. Aurora said Yes, you said that. 
 chandler said Yes, yes I did, but what I didn't say was what I was about to say, what I wanted to say was, uh would you like to go out with me sometime, thankyou, goodnight.


In [7]:
class ChatData(Dataset):

    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.X = []

        for i in np.array(self.data):
            self.X.append("<startofstring> " + str(i[0]) + " <bot>: " + str(i[1]) + " <endofstring>")

        self.X = self.X[ : -1]


        print(self.X[0])

        self.X_encoded = tokenizer(self.X,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [8]:
tokenizer.add_special_tokens( {
 "pad_token" : "<pad>",
 "bos_token" : "<startofstring>",
 "eos_token" : "<endofstring>"

} )


tokenizer.add_tokens("<bot>:")

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

chatData = ChatData(df_final, tokenizer)
chatData = DataLoader(chatData, batch_size = 64)

<startofstring>  in the episode The One with the Sonogram at the End the plot Ross finds out his ex-wife is pregnant. Rachel returns her engagement ring to Barry. Monica becomes stressed when her and Ross's parents come to visit. Chandler Bing said Oh, I think this is the episode of Three's Company where there's some kind of misunderstanding. <bot>: chandler said Oh, I think this is the episode of Three's Company where there's some kind of misunderstanding. <endofstring>


In [9]:
def train(chatData, model, optim):

    epochs = 12
    weight_decay=0.01
    for i in tqdm.tqdm(range(epochs)):
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "model_state.pt")
        print(infer("hello how are you"))

def infer(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a , max_new_tokens = 50)
    output = tokenizer.decode(output[0])
    return output


device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])
tokenizer.padding_side = "left"
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

# print(tokenizer.decode(model.generate(**tokenizer("hey i was good at basketball but ",
#                          return_tensors="pt"))[0]))

chatData = ChatData(df_final, tokenizer)
chatData =  DataLoader(chatData, batch_size=64 )

model.train()

optim = Adam(model.parameters(), lr=1e-3)

print("training .... ")
train(chatData, model, optim)



<startofstring>  in the episode The One with the Sonogram at the End the plot Ross finds out his ex-wife is pregnant. Rachel returns her engagement ring to Barry. Monica becomes stressed when her and Ross's parents come to visit. Chandler Bing said Oh, I think this is the episode of Three's Company where there's some kind of misunderstanding. <bot>: chandler said Oh, I think this is the episode of Three's Company where there's some kind of misunderstanding. <endofstring>
training .... 


  8%|▊         | 1/12 [01:38<18:03, 98.53s/it]

<startofstring> hello how are you  <bot>:   in the<endofstring> the episode The One with Rachel's Sister the plot Monica and Chandler decide to share their vows, but they decide to teach her sister to teach her what she wants. Monica and Chandler decide to get married and get married. Chandler and Chandler


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 17%|█▋        | 2/12 [03:10<15:49, 94.96s/it]

<startofstring> hello how are you  <bot>:   in the episode The One with Rachel's Work Laugh the plot Monica and Chandler's work Laugh the plot Chandler and Monica laugh at the piano bar and even find out Mike's parents. Monica and Chandler decide to share each other's secrets. Ross and Rachel


...

<startofstring> hello how are you  <bot>:   in the episode The One Where Rachel's Sister Babysits the plot Rachel's sister Amy comes to visit and moves in with her and Joey. Phoebe constantly and accidentally ruins Mike's apartment's wedding ring, Tag, during Ross's surprise birthday


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 92%|█████████▏| 11/12 [16:46<01:30, 90.70s/it]

<startofstring> hello how are you  <bot>:   in the episode The One with the Birthing Video the plot During Valentine's Day, Chandler discovers the miracle of birth seeing an extremely graphic video. Monica Geller said Okay. Ooh, Chandler, what's going? I mean to tell you are


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
100%|██████████| 12/12 [18:19<00:00, 91.61s/it]

<startofstring> hello how are you  <bot>:   in the episode The One with the One with the Birthing Video the plot During Valentine's Day, Chandler discovers the miracle of birth seeing an extremely graphic video. Monica Geller said Oh, my God!  <bot>:  chandler said Oh, Chandler


In [21]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import tqdm

# Assuming you have the original responses as references and generated responses as candidates
references = df_test['utterance_with_context'][:-1]  # Assuming you want to use the first 100 responses as a subset
candidates = []

# Generate responses using the model and collect them in the candidates list
for inp in tqdm.tqdm(references):
    generated_response = infer(inp)
    candidates.append(generated_response)

# Compute BLEU score
references_split = [[ref.split()] for ref in references]
candidates_split = [c.split() for c in candidates]

# BLEU score for the entire dataset
corpus_bleu_score = corpus_bleu(references_split, candidates_split)

print("Corpus BLEU score:", corpus_bleu_score)

100%|██████████| 856/856 [09:38<00:00,  1.48it/s]


Corpus BLEU score: 0.6332069737847894


In [19]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [22]:
!pip install rouge
from rouge import Rouge

rouge = Rouge()



scores = [rouge.get_scores(hyp, ref) for hyp, ref in zip(candidates, references)]

import numpy as np

# Initialize accumulators
rouge_1_f1 = []
rouge_2_f1 = []

# Aggregate scores
for score in scores:
    rouge_1_f1.append(score[0]['rouge-1']['f'])
    rouge_2_f1.append(score[0]['rouge-2']['f'])

# Calculate average
avg_rouge_1_f1 = np.mean(rouge_1_f1)
avg_rouge_2_f1 = np.mean(rouge_2_f1)

print("Average ROUGE-1 F1:", avg_rouge_1_f1)
print("Average ROUGE-2 F1:", avg_rouge_2_f1)

std_rouge_1_f1 = np.std(rouge_1_f1)
std_rouge_2_f1 = np.std(rouge_2_f1)

print("Standard Deviation ROUGE-1 F1:", std_rouge_1_f1)
print("Standard Deviation ROUGE-2 F1:", std_rouge_2_f1)





Average ROUGE-1 F1: 0.8916380889773173
Average ROUGE-2 F1: 0.8532703701372748
Standard Deviation ROUGE-1 F1: 0.03723908061868114
Standard Deviation ROUGE-2 F1: 0.04285135206408602
